<a href="https://colab.research.google.com/github/IlyaZelinskiy/Wav2vec_CFT_SHIFT_LAB/blob/main/Wav2Vec2_vec_embed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers==4.11.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 13.9 MB/s 
     |████████████████████████████████| 3.3 MB 45.9 MB/s 
     |████████████████████████████████| 880 kB 48.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=48bd9c87964d3c21ff3fa6a118144da19707da208fb69286db623e2c8be2d99c
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.2
    Uninstalling tokenizers-0.13.2:
      Successfully uninstalled tokenizers-0.13.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.25.0.dev0
    Uninstalling transformers-4.25.0.dev0:
      Successfully uninstalled transformers-4.25.0.dev0


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import torch
from datasets import load_dataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from torch.utils.data import Dataset, DataLoader

In [5]:
dataset = load_dataset("hf-internal-testing/librispeech_asr_demo",'clean', split="validation")
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset librispeech_asr_demo downloaded and prepared to /root/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_demo/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b. Subsequent calls will reuse this data.


In [ ]:
dataset['audio'][0]['array']

array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
       0.0010376 ], dtype=float32)

In [ ]:
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torchaudio
!pip install librosa

In [7]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model1 = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
#from transformers import AutoProcessor
from transformers import Wav2Vec2ForCTC

#processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-xls-r-300m")

model2 = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-xls-r-300m")

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-xls-r-300m were not used when initializing Wav2Vec2ForCTC: ['quantizer.weight_proj.weight', 'quantizer.codevectors', 'project_hid.bias', 'project_q.weight', 'quantizer.weight_proj.bias', 'project_q.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it 

In [10]:
import pandas as pd

In [17]:
def emb_to_plt(processor, model):
    array = []
    for i in range(73):
        inputs = processor(dataset["audio"][i]['array'], sampling_rate=sampling_rate, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states = True)
            array.append(outputs.hidden_states[22])   
    array2 = []
    for i in range(72):
        pca = PCA(n_components=15, random_state=42)
        emb_15d = pca.fit_transform(array[i][0])
        array2.append(emb_15d)
    for i in range(72):
        array2[i] = torch.tensor(array2[i])
    emb = array[0][0]
    embedd = array2[0]
    for i in range(1,72):
        emb = torch.cat((emb, array[i][0]), dim = 0)
        embedd = torch.cat((embedd, array2[i]), dim = 0)
    clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=0.6, affinity='cosine', linkage='average').fit(embedd)
    pca = PCA(n_components=2, random_state=42)
    emb_2d = pd.DataFrame(pca.fit_transform(emb), columns=['x1', 'x2'])
    emb_2d['label'] = clustering.labels_
    emb_2d['label'].nunique()
    import plotly.express as px
    fig = px.scatter(emb_2d, x='x1', y='x2', color='label', width=800, height=600)
    fig.show()

In [11]:
emb_to_plt(processor, model1)

ENCODDING 1 layer

In [18]:
emb_to_plt(processor, model2)